## Command imports

In [111]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
import glob
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


## Import mse IDs 

In [112]:
#subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
subjects = ["mse1343"]
print subjects[:5]

['mse1343']


## Set variables to lesion/segmentation files

In [129]:
for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[0]):
    print les_file
for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[0]):
    print seg_file
    
print seg_file, les_file

/data/henry7/PBR/subjects/mse1343/lesions_manual/ms69-mse1343-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1343/masks/ms69-mse1343-002-AX_T1_3D_IRSPGR/segmentation.nii.gz
/data/henry7/PBR/subjects/mse1343/masks/ms69-mse1343-002-AX_T1_3D_IRSPGR/segmentation.nii.gz /data/henry7/PBR/subjects/mse1343/lesions_manual/ms69-mse1343-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz


## Obtain affines 

In [114]:
les_img = nib.load(les_file)
les_img.dataobj
seg_img = nib.load(seg_file)
seg_img.dataobj

les_data = les_img.get_data()
print les_data.shape
seg_data = seg_img.get_data()
print seg_data.shape

les_affine = les_img.get_affine()
print les_affine
seg_affine = seg_img.get_affine()
print seg_affine
inv_seg_affine = np.linalg.inv(seg_affine)
print inv_seg_affine

(256, 256, 150)
(256, 256, 256)
[[  -0.9375        0.            0.          118.36100006]
 [   0.            0.9375        0.          -74.62950134]
 [   0.            0.            1.          -27.06119919]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          126.36100769]
 [   0.            0.            1.          -82.62950134]
 [   0.           -1.            0.          182.938797  ]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          126.36100769]
 [  -0.           -0.           -1.          182.938797  ]
 [   0.            1.            0.           82.62950134]
 [   0.            0.            0.            1.        ]]


In [115]:
#lesion labels
les_labels, n_les_labels = label(les_data==[1])

#segmentation labels
seg_brainstem_labels, n_seg_brainstem_labels = label(seg_data==[16])
seg_lcerebellumcortex_labels, n_seg_lcerebellumcortex_labels = label(seg_data==[8])
seg_rcerebellumcortex_labels, n_seg_rcerebellumcortex_labels = label(seg_data==[47])
seg_lcerebellumwm_labels, n_seg_lcerebellumwm_labels = label(seg_data==[7])
seg_rcerebellumwm_labels, n_seg_rcerebellumwm_labels = label(seg_data==[46])

In [116]:
n_les_labels

14

In [117]:
n_seg_brainstem_labels

2

In [118]:
n = np.unique(les_labels)
print n

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [119]:
np.bincount(les_labels.ravel())

array([9802412,      34,   10765,      30,      43,     962,     608,
         15063,      54,     159,      36,      89,      56,      33,
            56])

## Function that gets coordinates from labels

In [141]:
def get_label_coord(labels,num_labels):
    all_label_coords = []
    if num_labels >= 2:
        for count in range(1, num_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
    else:
        x,y,z = np.nonzero(labels==1)
        for count in range(len(x)):
            all_label_coords.append([x[count],y[count],z[count]])
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_label_coords

## Function that converts coordinates to real space

In [142]:
def get_rs_coord(coordinates):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(les_affine, [coordinates[count][count2][0],
                                           coordinates[count][count2][1], 
                                           coordinates[count][count2][2],
                                           1])
            rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

## Function that converts coordinates to segmentation file coordinates

In [143]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                coordinates[count][count2][1], 
                                                coordinates[count][count2][2],
                                                1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            #seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

## Generate midbrain coordinates

In [162]:
midbrain_int = []
midbrain_int.append(get_label_coord(seg_brainstem_labels,n_seg_brainstem_labels))
midbrain_int.append(get_label_coord(seg_lcerebellumcortex_labels,n_seg_lcerebellumcortex_labels))
midbrain_int.append(get_label_coord(seg_lcerebellumwm_labels,n_seg_lcerebellumwm_labels))
midbrain_int.append(get_label_coord(seg_rcerebellumcortex_labels,n_seg_rcerebellumcortex_labels))
midbrain_int.append(get_label_coord(seg_rcerebellumwm_labels,n_seg_rcerebellumwm_labels))

if len(midbrain_int) != 1:
    midbrain[0] = midbrain_int[1]
else:
    midbrain[0] = midbrain_int[0]

print len(midbrain[0])
print midbrain[0][4800]
    

56353
[134, 188, 73]


## Generate lesion coordinates into a variable

In [145]:
lesions = get_seg_coord(get_rs_coord(get_label_coord(les_labels,n_les_labels)))

print len(lesions)
print lesions[0][1]

14
[80, 133, 113]


## Function that takes in coordinates for lesions, and outputs average; may not actually use this

In [147]:
def average_func(coordinates):
    averages_all = []
    for count in range(len(coordinates)):
        averages = []
        sumx=0;sumy=0;sumz=0
        for count2 in range(len(coordinates[count])):
            sumx += coordinates[count][count2][0]
            sumy += coordinates[count][count2][1]
            sumz += coordinates[count][count2][2]
        average_x = sumx / len(coordinates[count])
        average_y = sumy / len(coordinates[count])
        average_z = sumz / len(coordinates[count])
        averages = [average_x, average_y, average_z]
        averages_all.append(averages)
        #print count, averages
    return averages_all

In [160]:
average_les = average_func(get_label_coord(les_labels, n_les_labels))
average_rs = average_func(get_rs_coord(get_label_coord(les_labels, n_les_labels)))
average_seg = average_func(get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels))))

allcoord_les = get_label_coord(les_labels, n_les_labels)
allcoord_rs = get_rs_coord(get_label_coord(les_labels, n_les_labels))
allcoord_seg = get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels)))

print average_les[0]
print average_rs[0]
print average_seg[0]
print allcoord_les[0][0]
print allcoord_rs[0][0]
print allcoord_seg[0][0]

print average_seg

[78, 113, 75]
[44.54666182574104, 31.804322186638327, 48.70350669412052]
[81, 133, 113]
[77, 113, 75]
[46.173500061035156, 31.307998657226562, 47.938800811767578]
[80, 134, 113]
[[81, 133, 113], [102, 126, 88], [97, 138, 160], [99, 158, 97], [107, 133, 152], [142, 135, 151], [158, 133, 87], [142, 176, 98], [143, 125, 141], [142, 148, 148], [147, 152, 53], [149, 148, 149], [152, 122, 144], [180, 108, 120]]


## Infratentorial Lesion Check

In [167]:
les_type = []
inf_num = 0
for count in range(len(lesions)):
    lesion_type = ""
    count2 = 0
    while lesion_type == "":
        if average_seg[count] == midbrain[0][count2]:
            inf_num += 1
            lesion_type = "infratentorial"
            count2 = len(midbrain[0])
        if count2 >= len(midbrain[0]):
            break
        else:
            count2 += 1
    if lesion_type == "infratentorial":
        les_type.append(lesion_type)
    elif lesion_type == "":
        les_type.append("subcortical")

print les_type
print inf_num

IndexError: list index out of range

### Not part of program, but this worked for one set of coordinates so loop data is compared to this

In [149]:
realspace_coordinates = np.dot(les_affine, [average_les[0][0],
                 average_les[0][1], 
                 average_les[0][2],
                 1])
average_les[0] 
realspace_coordinates
from numpy.linalg import inv
inv_seg_affine = np.linalg.inv(seg_affine)
freesurfer_matrix = np.dot(inv_seg_affine, [realspace_coordinates[0],
                                            realspace_coordinates[1], 
                                            realspace_coordinates[2],
                                            1])
print average_les[0]; print realspace_coordinates; print freesurfer_matrix

[78, 113, 75]
[ 45.23600006  31.30799866  47.93880081   1.        ]
[  81.12500763  134.99999619  113.9375        1.        ]


# Master file

for subject_count in range(len(subjects)):
    #show file directory
    for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[subject_count]):
        print les_file_1328
    for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[subject_count]):
        print seg_file_1328
    #loadfile
    les_img = nib.load(les_file)
    les_img.dataobj
    seg_img = nib.load(seg_file)
    seg_img.dataobj
    #print shape
    les_data = les_img.get_data()
    print les_data.shape
    seg_data = seg_img.get_data()
    print seg_data.shape
    #print affines
    les_affine = les_img.get_affine()
    print les_affine
    seg_affine = seg_img.get_affine()
    print seg_affine
    inv_seg_affine = np.linalg.inv(seg_affine)
    print inv_seg_affine
    #set labels
    #lesion labels
    les_labels, nles_labels = label(les_data==[1])
    #segmentation labels
    seg_les_labels, seg_nles_labels = label(seg_data==[4])
    #get coordinates from lesion
    def get_label_coord():
        all_label_coords = []
        for count in range(1, nles_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(les_labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
        #for count in range(nlabels):
        #    print "lesion #", count+1, "", all_label_coords[count]
        #    print ""
        return all_label_coords
    #get coordinates from lesion
    def get_rs_coord(coordinates):
        all_rs_coords = []
        for count in range(len(coordinates)):
            cur_rs_coords = []
            for count2 in range(len(coordinates[count])):
                rs_coord = np.dot(les_affine, [coordinates[count][count2][0],
                                                   coordinates[count][count2][1], 
                                                   coordinates[count][count2][2],
                                                   1])
                rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
                #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
                #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
                cur_rs_coords.append(rs_coord_noone)
            all_rs_coords.append(cur_rs_coords)
        #for count in range(nlabels):
        #    print "lesion #", count+1, "", all_rs_coords[count]
        #    print ""
        return all_rs_coords
    #get coordinates from real world
    def get_seg_coord(coordinates):
        all_seg_coords = []
        for count in range(len(coordinates)):
            cur_seg_coords = []
            for count2 in range(len(coordinates[count])):
                seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                        coordinates[count][count2][1], 
                                                        coordinates[count][count2][2],
                                                        1])
                #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
                seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
                #seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
                cur_seg_coords.append(seg_coord_noone)
            all_seg_coords.append(cur_seg_coords)
        #for count in range(nlabels):
        #    print "lesion #", count+1, "", all_seg_coords[count]
        #    print ""
        return all_seg_coords